In [1]:
import torch as tc
import pandas as pd
import numpy as np
from src.bbox import BBox
import open3d as otd
from tqdm import tqdm
import matplotlib.pyplot as plt
otd_vector3d = otd.utility.Vector3dVector
from src.star.star import STAR
from src.curve_utils import CurveUtils
from src.curve_generator import CurveGenerator
from src.mesh_manipulation import save_obj
device = tc.device("cuda" if tc.cuda.is_available() else "cpu")
genders = ['female', 'male']
male = ['male']
female = ['female']

In [2]:
their_semantics = [
    'Bust girth',
    'Waist girth',
    'Hip girth',
    'Thigh girth R',
    'Upper arm girth R',
    'Neck girth',
    'Back neck point to waist',
    'Height (m)',
]
our_semantic = [
    'bust_chest_girth', # 5.3.4
    'waist_girth', # 5.3.10
    'hip_girth', # 5.3.13
    'thigh_girth', # 5.3.20
    'upper_arm_girth', # 5.3.16
    'neck_girth', # 5.3.2
    'neck_waist',
    'stature', # 5.1.1
]

curve_index = {
    'neck_girth':4, # 5.3.2
    'bust_chest_girth': 0, # 5.3.4
    'waist_girth': 1, # 5.3.10
    'hip_girth': 1, # 5.3.13
    'upper_arm_girth': 3, # 5.3.16
    'thigh_girth': 2, # 5.3.20
}

In [3]:
subdivided_bodies = tc.load('data/subdivided_bodies.pt')
measures = pd.read_pickle(f'data/cleaned_measures.zip')
measures.index = measures['Subject']
measures = measures[measures['Measuring station'] == "MOVE4D"]

mfd_gender_measures = dict()
for gender in genders:
    mfd_gender_measures[gender] = measures[measures['Sex'] == gender]
    mfd_gender_measures[gender] = mfd_gender_measures[gender][their_semantics]
    mfd_gender_measures[gender].columns = our_semantic
    mfd_gender_measures[gender]['stature'] *= 1000

In [4]:
selected_subjects = dict()
selected_measures = dict()

for gender in genders:
    gender_measures = measures[measures['Sex'] == gender]
    temp_measures = gender_measures[their_semantics].iloc[::2]
    selected_subjects[gender] = 'IEEEP2_07' if gender == 'female' else 'IEEEP2_04'
    selected_measures[gender] = temp_measures.loc[selected_subjects[gender]]
    selected_subjects[gender] = temp_measures.index.get_loc(selected_subjects[gender])*2
    selected_measures[gender].index = our_semantic
    selected_measures[gender]['stature'] *= 1000

In [5]:
body = subdivided_bodies['vertices']['female'][selected_subjects['female']]
template = subdivided_bodies['faces']['female']+1

In [6]:
gender_curves = dict()
for gender in genders:
    print(f'SEGMENTING {gender.upper()} BODIES', end='')
    faces = subdivided_bodies['faces'][gender].to(device)
    bodies = subdivided_bodies['vertices'][gender]
    body = bodies[selected_subjects[gender]].to(device)
    measures = selected_measures[gender]
    result = CurveGenerator.get_curves(body, measures, faces, device, gender)
    gender_curves[gender] = result[0]
tc.save(gender_curves, "data/gender_curves.zip")

SEGMENTING FEMALE BODIES

processing body:  98%|█████████▊| 44/45 [06:51<00:09,  9.35s/it]


SEGMENTING MALE BODIES

processing body:  98%|█████████▊| 44/45 [07:51<00:10, 10.72s/it]


In [7]:
gender_measures = dict()
for gender in genders:
    print(f'MEASURING {gender.upper()} BODIES', end='')
    gender_measures[gender] = [[]]*5
    for segments_index, curves_segments in enumerate(gender_curves[gender]):
        gender_measures[gender][segments_index] = []
        for curves_index, curves in enumerate(tqdm(curves_segments)): # bust, torso, leg, arm, neck
            
            gender_measures[gender][segments_index].append([])
            for body in subdivided_bodies['vertices'][gender]:
                position = CurveUtils.generate_positions(tc.FloatTensor(curves).to(device), body.to(device))
                calculated_measures = CurveUtils.calculate_distances(position)
                gender_measures[gender][segments_index][curves_index].append(calculated_measures)

            calculated_measures = gender_measures[gender][segments_index][curves_index]
            gender_measures[gender][segments_index][curves_index] = tc.FloatTensor(calculated_measures)
        gender_measures[gender][segments_index] = tc.row_stack(gender_measures[gender][segments_index])

tc.save(gender_measures, "data/calculated_measures.zip")


MEASURING FEMALE BODIES

100%|██████████| 6040/6040 [07:17<00:00, 13.81it/s]


MEASURING MALE BODIES

100%|██████████| 6760/6760 [08:07<00:00, 13.87it/s]


In [10]:
best_gender_measures = dict() ## caso médio
for idx, gender in enumerate(genders):
    best_gender_measures[gender] = []
    for index, curve in enumerate(our_semantic[:-2]):
        measured = gender_measures[gender][index].T[::2].T
        ground_truth = mfd_gender_measures[gender][curve][::2]
        result = (measured - tc.FloatTensor(ground_truth/10).unsqueeze(0)).abs()
        min_rows_values = result.mean(1)
        min_columns_values, min_columns_indices = min_rows_values.min(0)
        best = min_columns_indices
        best_gender_measures[gender].append((
            best.numpy(),
            result[best].min().numpy(),
            result[best].max().numpy(),
            result[best].mean().numpy(),
            result[best].std().numpy()
        ))
gender_results = {
    'male': pd.DataFrame(best_gender_measures['male'], columns=['best','min', 'max', 'mean', 'std'], index=our_semantic[:-2]),
    'female': pd.DataFrame(best_gender_measures['female'], columns=['best','min', 'max', 'mean', 'std'], index=our_semantic[:-2])
}
print("male errors:")
print(gender_results['male'])
print("\nfemale errors:")
print(gender_results['female'])

tc.save(gender_results, "data/best_gender_curves.zip")

male errors:
                  best           min         max        mean         std
bust_chest_girth   187   0.050933838   4.6888275   1.1817387   1.0458738
waist_girth        335   0.048301697   1.6495056   0.5694864   0.4000256
hip_girth          132   0.007156372   1.2815323   0.4521351   0.3367785
thigh_girth        543   0.037849426   1.9917145  0.68962806   0.5091672
upper_arm_girth    390  0.0028152466  0.90377045    0.312722  0.20489208
neck_girth        5268    0.01726532   2.3889008   0.6928124   0.5737223

female errors:
                  best           min        max        mean         std
bust_chest_girth   147   0.020835876   4.166733   1.7483453   1.2649621
waist_girth        301  0.0072021484  3.5882034   1.0795076    0.954025
hip_girth          109   0.065208435  4.9380493   1.1920289   1.0881943
thigh_girth        472    0.05659485   3.587326  0.82730645   0.7422466
upper_arm_girth    346   0.014976501  1.3496666  0.54173285  0.35838273
neck_girth        3800  0.00

In [11]:
best_gender_curves = dict()
for gender in genders:
    all_positions = []
    best_gender_curves[gender] = []
    for index, curve in enumerate(our_semantic[:-2]):
        best = gender_results[gender].loc[curve]['best']
        coordinates = gender_curves[gender][curve_index[curve]][best]
        best_gender_curves[gender].append(coordinates)
        faces = subdivided_bodies['faces'][gender].to(device)
        bodies = subdivided_bodies['vertices'][gender]
        body = bodies[29].to(device)
        position = CurveUtils.generate_positions(tc.FloatTensor(coordinates).to(device), body.to(device))
        all_positions.append(position)
tc.save(best_gender_curves, 'data/selected_gender_curves.zip')